In [15]:
%matplotlib inline
from __future__ import print_function
import os,sys
notebook_code_root = os.path.dirname(os.getcwd())
data_root = os.path.dirname(notebook_code_root) + '/data/'
print("Data root: %s" % data_root)
sys.path.insert(0,notebook_code_root)

import cutils
from cutils import *

from keras.layers.core import Reshape
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

from vgg16 import *

opts = NotebookData(data_dir=data_root, sample_mode=False, preprocess=True)
print(opts)

Data root: /home/ubuntu/data/
Options:
  Testing directory: /home/ubuntu/data/statefarm/test/
  Training directory: /home/ubuntu/data/statefarm/train/
  Validation directory: /home/ubuntu/data/statefarm/valid/
  Preprocess directory: /home/ubuntu/data/statefarm/preprocessed/
  Results directory: /home/ubuntu/data/statefarm/results


In [2]:
# Load the VGG model.
vgg = Vgg16()

In [3]:
ls = vgg.model.layers
idx = 0
for i in range(len(ls)):
    if type(ls[i]) is MaxPooling2D:
        idx = i

conv_model = Sequential(layers=ls[:(idx+1)])
fc_layers = ls[(idx+1):]

In [4]:
def get_data(opt):
    tf = None
    vf = None
    tc = None
    vc = None
    trn_dir = os.path.join(opt.data_root, 'preprocessed', 'conv')
    if opt.preprocess:
        gen = image.ImageDataGenerator()
        img_trn_batches = gen.flow_from_directory(opt.train_dir(),
                                                  target_size=(224,224),
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=64)
        img_val_batches = gen.flow_from_directory(opt.valid_dir(),
                                                  target_size=(224,224),
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=64)
        tc = to_categorical(img_trn_batches.classes)
        vc = to_categorical(img_val_batches.classes)
        tf = conv_model.predict_generator(img_trn_batches, verbose=1)
        vf = conv_model.predict_generator(img_val_batches, verbose=1)
        if not os.path.exists(trn_dir):
            os.makedirs(trn_dir)
        save_array(os.path.join(trn_dir, 'train-dt'), tf)
        save_array(os.path.join(trn_dir, 'val-dt'), vf)
        save_array(os.path.join(trn_dir, 'train-cl'), tc)
        save_array(os.path.join(trn_dir, 'val-cl'), vc)
    else:
        tf = load_array(os.path.join(trn_dir, 'train-dt'))
        vf = load_array(os.path.join(trn_dir, 'val-dt'))
        tc = load_array(os.path.join(trn_dir, 'train-cl'))
        vc = load_array(os.path.join(trn_dir, 'val-cl'))
        
    return (tf, vf, tc, vc)

In [5]:
trn_features, val_features, img_trn_classes, img_val_classes = get_data(opts)

Found 17622 images belonging to 10 classes.
Found 4802 images belonging to 10 classes.
76/76 [==============================] - 92s 1s/step


In [6]:
def rand_training(trn, img):
    zipped_trn = zip(trn, img)
    np.random.shuffle(zipped_trn)
    trn_data = np.stack([data for data, cl in zipped_trn], axis=0)
    trn_cls = np.stack([cl for data, cl in zipped_trn], axis=0)
    return trn_data, trn_cls

In [103]:
# Create a new model with only dense layers.
fc_model = Sequential([
     # Input shape is shape of the conv result.
     Reshape((25088,), input_shape=(512, 7, 7,)),
     Dense(64, activation="relu"),
     BatchNormalization(),
     Dropout(0.2),
     Dense(64, activation='relu'),
     BatchNormalization(),
     Dropout(0.3),
     Dense(32, activation='relu'),
     BatchNormalization(),
     Dropout(0.6),
     Dense(10, activation='softmax')
    ])

In [104]:
fc_model.compile(optimizer=Adam(lr=0.000002),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [105]:
for i in range(40):
    training_data, training_classes = rand_training(trn_features, img_trn_classes)
    fc_model.fit(x=training_data,
                 y=training_classes,
                 batch_size=32,
                 validation_data=(val_features,img_val_classes),
                 epochs=2,
                 verbose=1)

Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 198us/step - loss: 3.3485 - acc: 0.1167 - val_loss: 2.4976 - val_acc: 0.1487
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 3.1205 - acc: 0.1352 - val_loss: 2.4364 - val_acc: 0.1710
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 2.8971 - acc: 0.1616 - val_loss: 2.3905 - val_acc: 0.1908
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 2.7344 - acc: 0.1874 - val_loss: 2.3390 - val_acc: 0.2039
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 192us/step - loss: 2.5712 - acc: 0.2172 - val_loss: 2.3061 - val_acc: 0.2137
Epoch 2/2
17622/17622 [==============================] - 3s 192us/step - loss: 2.4254 - acc: 0.2426 - val_loss: 2.2626 - val_acc: 0.2268
Train on 17622 samples, validat

In [106]:
for i in range(40):
    training_data, training_classes = rand_training(trn_features, img_trn_classes)
    fc_model.fit(x=training_data,
                 y=training_classes,
                 batch_size=32,
                 validation_data=(val_features,img_val_classes),
                 epochs=2,
                 verbose=1)

Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 4s 199us/step - loss: 0.6351 - acc: 0.8150 - val_loss: 1.4023 - val_acc: 0.5546
Epoch 2/2
17622/17622 [==============================] - 3s 192us/step - loss: 0.6321 - acc: 0.8149 - val_loss: 1.3989 - val_acc: 0.5464
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.6273 - acc: 0.8170 - val_loss: 1.4033 - val_acc: 0.5498
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.6285 - acc: 0.8159 - val_loss: 1.3909 - val_acc: 0.5462
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.6036 - acc: 0.8281 - val_loss: 1.3821 - val_acc: 0.5541
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.6197 - acc: 0.8157 - val_loss: 1.3718 - val_acc: 0.5577
Train on 17622 samples, validat

In [ ]:
# fc_model.compile(optimizer=Adam(lr=0.000001),
#                  loss='categorical_crossentropy',
#                  metrics=['accuracy'])
for i in range(40):
    training_data, training_classes = rand_training(trn_features, img_trn_classes)
    fc_model.fit(x=training_data,
                 y=training_classes,
                 batch_size=32,
                 validation_data=(val_features,img_val_classes),
                 epochs=2,
                 verbose=1)

Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 4s 201us/step - loss: 0.3230 - acc: 0.9306 - val_loss: 1.2013 - val_acc: 0.6283
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.3169 - acc: 0.9326 - val_loss: 1.2046 - val_acc: 0.6297
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.3139 - acc: 0.9377 - val_loss: 1.1985 - val_acc: 0.6337
Epoch 2/2
17622/17622 [==============================] - 3s 194us/step - loss: 0.3122 - acc: 0.9354 - val_loss: 1.2070 - val_acc: 0.6231
Train on 17622 samples, validate on 4802 samples
Epoch 1/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.3153 - acc: 0.9332 - val_loss: 1.2041 - val_acc: 0.6289
Epoch 2/2
17622/17622 [==============================] - 3s 193us/step - loss: 0.3169 - acc: 0.9301 - val_loss: 1.2105 - val_acc: 0.6277
Train on 17622 samples, validat

In [101]:
process_model(fc_model, opts, 'v1', sub=True)

Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch0
10000/10000 [==============================] - 1s 125us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch1
10000/10000 [==============================] - 1s 112us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch2
10000/10000 [==============================] - 1s 110us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch3
10000/10000 [==============================] - 1s 111us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch4
10000/10000 [==============================] - 1s 110us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch5
10000/10000 [==============================] - 1s 112us/step
Predicting batch at /home/ubuntu/data/statefarm/preprocessed//test/batch6
10000/10000 [==============================] - 1s 110us/step
Predicting batch at /home/ubuntu/data/statefarm/preproc

In [11]:
from IPython.display import FileLink

FileLink('submission.csv')

/home/ubuntu/nbs/lesson-3/submission.csv